这里很多的内容是 书上 11章的

feature_training.cpp  这个是特征值字典的词袋问题训练
调用的是对应的库


In [2]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import glob
import os
import pickle

def main():
    print("Reading images...")
    images = []
    image_files = glob.glob("./data/*.png")
    for file in sorted(image_files):
        img = cv2.imread(file)
        if img is not None:
            images.append(img)
        else:
            print(f"Failed to read image: {file}")

    if not images:
        print("No images found. Please check the data directory.")
        return

    print(f"Loaded {len(images)} images.")

    print("Detecting ORB features...")
    detector = cv2.ORB_create()
    descriptors = []

    for image in images:
        keypoints, descriptor = detector.detectAndCompute(image, None)
        if descriptor is not None:
            descriptors.append(descriptor)
        else:
            print(f"No features detected in an image.")

    if not descriptors:
        print("No features detected in any image. Cannot create vocabulary.")
        return

    # Flatten the list of descriptors
    all_descriptors = np.vstack(descriptors)

    print("Creating vocabulary...")
    n_words = 1000  # You can adjust this number
    kmeans = KMeans(n_clusters=n_words, n_init=10)
    kmeans.fit(all_descriptors)

    print("Vocabulary info:")
    print(f"Number of visual words: {n_words}")
    print(f"Feature dimension: {all_descriptors.shape[1]}")

    output_file = "vocabulary.pkl"
    with open(output_file, 'wb') as f:
        pickle.dump(kmeans, f)
    print(f"Vocabulary saved to {output_file}")

    print("Done")

if __name__ == "__main__":
    main()



Reading images...
Loaded 10 images.
Detecting ORB features...
Creating vocabulary...
Vocabulary info:
Number of visual words: 1000
Feature dimension: 32
Vocabulary saved to vocabulary.pkl
Done


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import glob
import os
import pickle
from tqdm import tqdm

def read_associate_file(filename):
    rgb_files, depth_files = [], []
    rgb_times, depth_times = [], []
    
    with open(filename, 'r') as f:
        for line in f:
            if len(line) == 0 or line[0] == '#':
                continue
            data = line.split()
            rgb_times.append(float(data[0]))
            rgb_files.append(data[1])
            depth_times.append(float(data[2]))
            depth_files.append(data[3])
    
    return rgb_files, depth_files, rgb_times, depth_times

def extract_features(image_files, dataset_dir):
    detector = cv2.ORB_create()
    descriptors = []
    
    for image_file in tqdm(image_files, desc="Extracting features"):
        image = cv2.imread(os.path.join(dataset_dir, image_file))
        if image is None:
            print(f"Failed to read image: {image_file}")
            continue
        
        keypoints, descriptor = detector.detectAndCompute(image, None)
        if descriptor is not None:
            descriptors.append(descriptor)
    
    return np.vstack(descriptors)

def create_vocabulary(descriptors, num_words=4500): #这里要改一下数据里面的词没这么多
    print("Creating vocabulary...")
    kmeans = KMeans(n_clusters=num_words, n_init=1, max_iter=50, verbose=1)
    kmeans.fit(descriptors)
    return kmeans

def main():
    dataset_dir = "/Users/bytedance/Desktop/test/slambook_python/ch8/data/" #在ch8里面
    associate_file = os.path.join(dataset_dir, "associate.txt")
    
    if not os.path.exists(associate_file):
        print("Please generate the associate file called associate.txt!")
        return

    rgb_files, _, _, _ = read_associate_file(associate_file)
    
    print("Extracting features...")
    all_descriptors = extract_features(rgb_files, dataset_dir)
    print(f"Extracted {all_descriptors.shape[0]} features.")

    vocabulary = create_vocabulary(all_descriptors)

    print("Vocabulary info:")
    print(f"Number of visual words: {vocabulary.n_clusters}")
    print(f"Feature dimension: {all_descriptors.shape[1]}")

    output_file = "vocab_larger.pkl"
    with open(output_file, 'wb') as f:
        pickle.dump(vocabulary, f)
    print(f"Vocabulary saved to {output_file}")

    print("Done")

if __name__ == "__main__":
    main()



Extracting features...


Extracting features:   0%|          | 0/573 [00:00<?, ?it/s][ WARN:0@3775.758] global loadsave.cpp:241 findDecoder imread_('/Users/bytedance/Desktop/test/slambook_python/ch8/data/rgb/1305031453.659600.png'): can't open/read file: check file path/integrity
[ WARN:0@3775.758] global loadsave.cpp:241 findDecoder imread_('/Users/bytedance/Desktop/test/slambook_python/ch8/data/rgb/1305031453.691678.png'): can't open/read file: check file path/integrity
[ WARN:0@3775.759] global loadsave.cpp:241 findDecoder imread_('/Users/bytedance/Desktop/test/slambook_python/ch8/data/rgb/1305031453.727652.png'): can't open/read file: check file path/integrity
[ WARN:0@3775.759] global loadsave.cpp:241 findDecoder imread_('/Users/bytedance/Desktop/test/slambook_python/ch8/data/rgb/1305031453.759694.png'): can't open/read file: check file path/integrity
[ WARN:0@3775.759] global loadsave.cpp:241 findDecoder imread_('/Users/bytedance/Desktop/test/slambook_python/ch8/data/rgb/1305031453.791716.png'): can't op

Failed to read image: rgb/1305031453.659600.png
Failed to read image: rgb/1305031453.691678.png
Failed to read image: rgb/1305031453.727652.png
Failed to read image: rgb/1305031453.759694.png
Failed to read image: rgb/1305031453.791716.png
Failed to read image: rgb/1305031453.827773.png
Failed to read image: rgb/1305031453.859705.png
Failed to read image: rgb/1305031453.891710.png
Failed to read image: rgb/1305031453.927723.png
Failed to read image: rgb/1305031453.959641.png
Failed to read image: rgb/1305031453.991624.png
Failed to read image: rgb/1305031454.027662.png
Failed to read image: rgb/1305031454.059654.png
Failed to read image: rgb/1305031454.091651.png
Failed to read image: rgb/1305031454.127701.png
Failed to read image: rgb/1305031454.159672.png
Failed to read image: rgb/1305031454.191658.png
Failed to read image: rgb/1305031454.227671.png
Failed to read image: rgb/1305031454.259627.png
Failed to read image: rgb/1305031454.291639.png
Failed to read image: rgb/1305031454.327

Initialization complete
Iteration 0, inertia 0.0.
Converged at iteration 0: center shift 0.0 within tolerance 0.5239422882413582.
Vocabulary info:
Number of visual words: 4500
Feature dimension: 32
Vocabulary saved to vocab_larger.pkl
Done


In [15]:
import cv2
import numpy as np
from sklearn.cluster import MiniBatchKMeans
import os
import yaml
import gzip
from tqdm import tqdm

def read_associate_file(filename):
    rgb_files, depth_files = [], []
    rgb_times, depth_times = [], []
    
    with open(filename, 'r') as f:
        for line in f:
            if len(line) == 0 or line[0] == '#':
                continue
            data = line.split()
            rgb_times.append(float(data[0]))
            rgb_files.append(data[1])
            depth_times.append(float(data[2]))
            depth_files.append(data[3])
    
    return rgb_files, depth_files, rgb_times, depth_times

def extract_features(image_files, dataset_dir):
    detector = cv2.ORB_create()
    descriptors = []
    
    for image_file in tqdm(image_files, desc="Extracting features"):
        image = cv2.imread(os.path.join(dataset_dir, image_file))
        if image is None:
            print(f"Failed to read image: {image_file}")
            continue
        
        keypoints, descriptor = detector.detectAndCompute(image, None)
        if descriptor is not None:
            descriptors.append(descriptor)
    
    return np.vstack(descriptors)

def create_vocabulary(descriptors, num_words=None):
    print("Creating vocabulary...")
    if num_words is None:
        num_words = min(len(descriptors) // 10, 1000000)  # 使用特征数量的1/10，但不超过1,000,000
    print(f"Creating vocabulary with {num_words} words...")
    
    kmeans = MiniBatchKMeans(n_clusters=num_words, batch_size=1000, n_init=3, max_iter=50, verbose=1)
    kmeans.fit(descriptors)
    return kmeans

def save_vocabulary_yml_gz(vocabulary, filename):
    # 创建一个字典来存储词汇信息
    vocab_data = {
        "n_clusters": vocabulary.n_clusters,
        "centers": vocabulary.cluster_centers_.tolist(),
        "n_features": vocabulary.cluster_centers_.shape[1]
    }

    # 将字典转换为YAML格式
    yaml_data = yaml.dump(vocab_data)

    # 使用gzip压缩并保存
    with gzip.open(filename, 'wt') as f:
        f.write(yaml_data)

def main():
    dataset_dir = "/Users/bytedance/Desktop/test/slambook_python/ch8/data/"
    associate_file = os.path.join(dataset_dir, "associate.txt")
    
    if not os.path.exists(associate_file):
        print("Please generate the associate file called associate.txt!")
        return

    rgb_files, _, _, _ = read_associate_file(associate_file)
    
    print("Extracting features...")
    all_descriptors = extract_features(rgb_files, dataset_dir)
    print(f"Extracted {all_descriptors.shape[0]} features.")

    vocabulary = create_vocabulary(all_descriptors)

    print("Vocabulary info:")
    print(f"Number of visual words: {vocabulary.n_clusters}")
    print(f"Feature dimension: {all_descriptors.shape[1]}")

    output_file = "vocab_larger.yml.gz"
    save_vocabulary_yml_gz(vocabulary, output_file)
    print(f"Vocabulary saved to {output_file}")

    print("Done")

if __name__ == "__main__":
    main()


Extracting features...


Extracting features:   0%|          | 0/573 [00:00<?, ?it/s][ WARN:0@8482.445] global loadsave.cpp:241 findDecoder imread_('/Users/bytedance/Desktop/test/slambook_python/ch8/data/rgb/1305031453.659600.png'): can't open/read file: check file path/integrity
[ WARN:0@8482.445] global loadsave.cpp:241 findDecoder imread_('/Users/bytedance/Desktop/test/slambook_python/ch8/data/rgb/1305031453.691678.png'): can't open/read file: check file path/integrity
[ WARN:0@8482.445] global loadsave.cpp:241 findDecoder imread_('/Users/bytedance/Desktop/test/slambook_python/ch8/data/rgb/1305031453.727652.png'): can't open/read file: check file path/integrity
[ WARN:0@8482.445] global loadsave.cpp:241 findDecoder imread_('/Users/bytedance/Desktop/test/slambook_python/ch8/data/rgb/1305031453.759694.png'): can't open/read file: check file path/integrity
[ WARN:0@8482.445] global loadsave.cpp:241 findDecoder imread_('/Users/bytedance/Desktop/test/slambook_python/ch8/data/rgb/1305031453.791716.png'): can't op

Failed to read image: rgb/1305031453.659600.png
Failed to read image: rgb/1305031453.691678.png
Failed to read image: rgb/1305031453.727652.png
Failed to read image: rgb/1305031453.759694.png
Failed to read image: rgb/1305031453.791716.png
Failed to read image: rgb/1305031453.827773.png
Failed to read image: rgb/1305031453.859705.png
Failed to read image: rgb/1305031453.891710.png
Failed to read image: rgb/1305031453.927723.png
Failed to read image: rgb/1305031453.959641.png
Failed to read image: rgb/1305031453.991624.png
Failed to read image: rgb/1305031454.027662.png
Failed to read image: rgb/1305031454.059654.png
Failed to read image: rgb/1305031454.091651.png
Failed to read image: rgb/1305031454.127701.png
Failed to read image: rgb/1305031454.159672.png
Failed to read image: rgb/1305031454.191658.png
Failed to read image: rgb/1305031454.227671.png
Failed to read image: rgb/1305031454.259627.png
Failed to read image: rgb/1305031454.291639.png
Failed to read image: rgb/1305031454.327

Inertia for init 3/3: 245985430.0
[MiniBatchKMeans] Reassigning 95 cluster centers.
Minibatch step 1/225: mean batch inertia: 81774.207
Minibatch step 2/225: mean batch inertia: 75691.23400101592, ewa inertia: 75691.23400101592
Minibatch step 3/225: mean batch inertia: 68707.66277445301, ewa inertia: 72588.11414917726
Minibatch step 4/225: mean batch inertia: 67222.79134052161, ewa inertia: 70204.05602491347
Minibatch step 5/225: mean batch inertia: 61858.18512409423, ewa inertia: 66495.60416940614
Minibatch step 6/225: mean batch inertia: 61022.34720979361, ewa inertia: 64063.5859691784
Minibatch step 7/225: mean batch inertia: 61284.567991162505, ewa inertia: 62828.741277769426
Minibatch step 8/225: mean batch inertia: 60513.0536344812, ewa inertia: 61799.77542871889
Minibatch step 9/225: mean batch inertia: 59640.61096796119, ewa inertia: 60840.35998292564
Minibatch step 10/225: mean batch inertia: 58752.050935955675, ewa inertia: 59912.428835638384
[MiniBatchKMeans] Reassigning 6 c

In [16]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import os
import yaml
import gzip

class Vocabulary:
    def __init__(self, filename):
        with gzip.open(filename, 'rt') as f:
            data = yaml.safe_load(f)
        self.centers = np.array(data['centers'])
        self.n_clusters = data['n_clusters']

    def transform(self, descriptors):
        distances = np.linalg.norm(descriptors[:, np.newaxis] - self.centers, axis=2)
        return np.bincount(distances.argmin(axis=1), minlength=self.n_clusters)

    def score(self, v1, v2):
        return cosine_similarity(v1.reshape(1, -1), v2.reshape(1, -1))[0, 0]

class Database:
    def __init__(self, vocab):
        self.vocab = vocab
        self.bow_vectors = []

    def add(self, descriptors):
        self.bow_vectors.append(self.vocab.transform(descriptors))

    def query(self, descriptors, max_results):
        query_bow = self.vocab.transform(descriptors)
        scores = [self.vocab.score(query_bow, bow) for bow in self.bow_vectors]
        top_indices = np.argsort(scores)[::-1][:max_results]
        return [(idx, scores[idx]) for idx in top_indices]

def main():
    print("Reading vocabulary...")
    vocab = Vocabulary("./vocab_larger.yml.gz")

    print("Reading images...")
    images = []
    for i in range(10):
        path = f"./data/{i+1}.png"
        img = cv2.imread(path)
        if img is not None:
            images.append(img)
        else:
            print(f"Failed to read image: {path}")

    print("Detecting ORB features...")
    detector = cv2.ORB_create()
    descriptors = []
    for image in images:
        keypoints, descriptor = detector.detectAndCompute(image, None)
        descriptors.append(descriptor)

    print("Comparing images with images")
    for i in range(len(images)):
        v1 = vocab.transform(descriptors[i])
        for j in range(i, len(images)):
            v2 = vocab.transform(descriptors[j])
            score = vocab.score(v1, v2)
            print(f"Image {i} vs Image {j}: {score}")
        print()

    print("Comparing images with database")
    db = Database(vocab)
    for desc in descriptors:
        db.add(desc)

    for i, desc in enumerate(descriptors):
        results = db.query(desc, 4)
        print(f"Searching for image {i} returns:")
        for idx, score in results:
            print(f"  Image {idx}: score = {score}")
        print()

    print("Done.")

if __name__ == "__main__":
    main()


Reading vocabulary...
Reading images...
Detecting ORB features...
Comparing images with images
Image 0 vs Image 0: 0.9999999999999999
Image 0 vs Image 1: 0.5894276891098109
Image 0 vs Image 2: 0.6557866486843913
Image 0 vs Image 3: 0.6828858103865518
Image 0 vs Image 4: 0.60670784253099
Image 0 vs Image 5: 0.6357238618110126
Image 0 vs Image 6: 0.6764321268994538
Image 0 vs Image 7: 0.6547731796661513
Image 0 vs Image 8: 0.6562801784909019
Image 0 vs Image 9: 0.7010372372687652

Image 1 vs Image 1: 1.0
Image 1 vs Image 2: 0.7071772127085524
Image 1 vs Image 3: 0.6009682656945503
Image 1 vs Image 4: 0.49403416569766423
Image 1 vs Image 5: 0.6408992196096019
Image 1 vs Image 6: 0.6234082500032324
Image 1 vs Image 7: 0.6198163612492991
Image 1 vs Image 8: 0.6946598614649002
Image 1 vs Image 9: 0.6627193200008894

Image 2 vs Image 2: 1.0000000000000002
Image 2 vs Image 3: 0.6307738141010764
Image 2 vs Image 4: 0.5911819511894996
Image 2 vs Image 5: 0.6473897429937452
Image 2 vs Image 6: 0.